## Data Modeling
***

Notes
1. Dummy
2. Logreg

XGBClassifier, RandomForest

3. XGBClassifier, f2
3. XGBClassifier, f2-weighted
3. XGBClassifier, f2-SMOTE
3. XGBClassifier, f2-ADASYN

Randomize winner?

Gridsearch from there?

3. XGBClassifier, f2 - randomizedcv
3. XGBClassifier, f2-weighted - randomizedcv
3. XGBClassifier, f2-SMOTE - randomizedcv
3. XGBClassifier, f2, randomizedcv
3. XGBClassifier, f2-weighted, randomizedcv
3. XGBClassifier, f2, randomizedcv

In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline as ImPipeline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, log_loss, roc_auc_score, make_scorer, fbeta_score
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, train_test_split, RepeatedStratifiedKFold, StratifiedKFold, RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import pandas as pd

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, log_loss, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, train_test_split, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

df = pd.read_csv('data/Cardiovascular_Diseases_Risk_Prediction_Dataset.csv')
ordinal = ['General_Health', 'Checkup', 'Age_Category']
numeric = list(df.select_dtypes(exclude=object).columns)
categorical = list(df.select_dtypes(object).columns)
for i in ordinal:
    categorical.remove(i)
categorical.remove('Heart_Disease')

# Splitting the training from the validation data
# Making sure the split is stratefied given teh imbalance of our target variable
y = df['Heart_Disease']
X = df.drop('Heart_Disease',axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=12, stratify= y)

# Transforming the target variable into 1's and 0's
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


# Listing categories in order for each ordinal variable.
health = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']
check = [
    'Never', '5 or more years ago', 'Within the past 5 years',
    'Within the past 2 years', 'Within the past year'
]
age = [
    '18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
    '60-64', '65-69', '70-74', '75-79', '80+'
]

# Instantiating an OrdinalEncoder transformer to encode ordinal variables. 
oe = OrdinalEncoder(categories=[health, check, age])


# Instantiating a OneHotEncoder transformer to be used on the categorical varaibles. 
ohe = OneHotEncoder()

# Creating a column transformer to be used in a pipeline
col_transformer = ColumnTransformer(transformers=[
    ('oe', OrdinalEncoder(categories=[health, check, age]), ordinal),
    ('ohe', OneHotEncoder(), categorical)
], remainder="passthrough")

def model_scores(model_name, model, X_train, y_train, model_list= [], notes = ''):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)
    scoring = {'f2': make_scorer(fbeta_score, beta=2),
               'recall': 'recall',
               'accuracy':'accuracy',
               'f1':'f1',
               'roc_auc':'roc_auc',
               'precision': 'precision'}
    scores = cross_validate(model,
                           X_train,
                           y_train,
                           scoring = scoring,
                           cv=skf,
                           n_jobs=-1)
    time = round(scores['fit_time'].mean() + scores['score_time'].mean(),0)
    f2 = round(scores['test_f2'].mean(),4)*100
    recall = round(scores['test_recall'].mean(),4)*100
    accuracy = round(scores['test_accuracy'].mean(),4)*100
    f1 = round(scores['test_f1'].mean(),4)*100
    roc_auc = round(scores['test_roc_auc'].mean(),4)*100
    precision = round(scores['test_precision'].mean(),4)*100
    model_list.append([model_name, time, f2, recall, accuracy, f1, roc_auc, precision, notes])
    df = pd.DataFrame(model_list, columns=['model', 'time (in s)', 'f2', 'recall', 'accuracy', 'f1', 'roc_auc', 'precision', 'notes'])
    return model_list, df


### Dummy Model

In [2]:
pipe = Pipeline(steps=[('ct', col_transformer),
                      ('dm', DummyClassifier(strategy='most_frequent'))])
dummy_model = pipe.fit(X_train, y_train)
ml, df1 = model_scores('DummyClassifier', dummy_model, X_train, y_train, model_list= [], notes= "Dummy Model")
df1

,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,DummyClassifier,1.0,0.0,0.0,91.92,0.0,50.0,0.0,Dummy Model


### Baseline Model

In [3]:
pipe = Pipeline(steps=[('ct', col_transformer),
                       ('ss', StandardScaler()),
                      ('log_reg', LogisticRegression(solver='lbfgs', max_iter=1000, random_state=12))])
logreg_model = pipe.fit(X_train, y_train)
ml, df1 = model_scores('LogisticRegression', logreg_model, X_train, y_train, model_list= ml, notes= "Baseline Model")
df1

,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,DummyClassifier,1.0,0.0,0.00,91.92,0.00,50.00,0.00,Dummy Model
1,LogisticRegression,1.0,7.5,6.18,91.94,11.03,83.41,51.39,Baseline Model


### Model Iteration


In [5]:
pipe_rfc = Pipeline(steps=[('ct', col_transformer),
                       ('rfc', RandomForestClassifier(random_state=12))])
rfc_model = pipe_rfc.fit(X_train, y_train)
compare_list, compare_df = model_scores('RandomForestClassifier', rfc_model, X_train, y_train, model_list= [], notes= "Basic")

pipe_xgb = Pipeline(steps=[('ct', col_transformer),
                       ('xgb', XGBClassifier(random_state=12, tree_method= 'hist'))])
xgb_model = pipe_xgb.fit(X_train, y_train)
compare_list, compare_df = model_scores('XGBClassifier', xgb_model, X_train, y_train, model_list= compare_list, notes= "Basic")
compare_df


,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,RandomForestClassifier,31.0,5.34,4.38,91.81,7.96,80.85,43.76,Basic
1,XGBClassifier,4.0,6.54,5.38,91.87,9.66,82.94,47.73,Basic


In [11]:
# Choose between XGBClassifier Variants

# Basic
ml_xgb, df_xgb = model_scores('XGBClassifier', xgb_model, X_train, y_train, model_list = [], notes= "Basic")

# Weighted
pipe = Pipeline(steps=[('ct', col_transformer),
                       ('xgb', XGBClassifier(random_state=12,scale_pos_weight=283883/24971))])
xgb_weighted = pipe.fit(X_train, y_train)
ml_xgb, df_xgb = model_scores('XGBClassifier', xgb_weighted, X_train, y_train, model_list = ml_xgb, notes= "Weighted")

# SMOTE
pipe = ImPipeline(steps=[('ct', col_transformer),
                       ('ss', StandardScaler()),
                       ('sm',SMOTE(random_state=12)),
                       ('xgb', XGBClassifier(random_state=12))])
xgb_smote = pipe.fit(X_train, y_train)
ml_xgb, df_xgb = model_scores('XGBClassifier', xgb_smote, X_train, y_train, model_list = ml_xgb, notes= "SMOTE")

# ADASYN
pipe = ImPipeline(steps=[('ct', col_transformer),
                       ('ss', StandardScaler()),
                       ('sm',ADASYN(random_state=12)),
                       ('xgb', XGBClassifier(random_state=12))])
xgb_adasyn = pipe.fit(X_train, y_train)
ml_xgb, df_xgb = model_scores('XGBClassifier', xgb_adasyn, X_train, y_train, model_list = ml_xgb, notes= "ADASYN")
df_xgb


,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,XGBClassifier,4.0,6.54,5.38,91.87,9.66,82.94,47.73,Basic
1,XGBClassifier,20.0,49.27,75.49,74.52,32.39,82.41,20.62,Weighted
2,XGBClassifier,60.0,6.75,5.57,91.81,9.90,82.81,44.54,SMOTE
3,XGBClassifier,69.0,5.84,4.81,91.78,8.63,82.82,42.56,ADASYN


In [14]:
# Randomize the Weighted
pipe = Pipeline(steps=[('ct', col_transformer),
                       ('xbg', XGBClassifier(random_state=12, tree_method='hist', scale_pos_weight= 283883/24971))])

params = {
    'xbg__n_estimators': range(50,1000,50),
    'xbg__max_depth': range(1,15),
    'xbg__eta': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
    'xbg__colsample_bytree': np.linspace(0,1,50),
    'xbg__min_child_weight': range(1,10),
    'xbg__gamma': [0, 0.1, 1, 10, 100, 1000],
    'xbg__reg_alpha': [0, 0.01, 0.1, 1, 10],
    'xbg__reg_lambda':[0, 0.01, 0.1, 1, 10]
}
ftwo_scorer = make_scorer(fbeta_score, beta=2)

rs = RandomizedSearchCV(estimator= pipe, param_distributions= params,
                       n_iter=500, scoring=ftwo_scorer, n_jobs=-1, cv=5, random_state=12)
rand_search = rs.fit(X_train, y_train)
rand_model = rand_search.best_estimator_
ml, df1 = model_scores('XGBClassifier', rand_model, X_train, y_train, model_list= [], notes = 'Best Estimator from RS')
print(rand_search.best_params_)
df1


{'xbg__reg_lambda': 10, 'xbg__reg_alpha': 0.1, 'xbg__n_estimators': 650, 'xbg__min_child_weight': 1, 'xbg__max_depth': 4, 'xbg__gamma': 10, 'xbg__eta': 0.15, 'xbg__colsample_bytree': 0.42857142857142855}


,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,XGBClassifier,7.0,50.32,80.33,72.72,32.25,83.63,20.18,Best Estimator from RS


In [16]:
# GridSearching the winnfer for the final
xgb = XGBClassifier(reg_lambda= 10, 
              reg_alpha= 0.1,
              min_child_weight= 1,
              gamma= 10,
              colsample_bytree= 0.42857142857142855,
              tree_method= "hist",
              scale_pos_weight=283883/24971,
              random_state=12)

pipe = Pipeline(steps=[('ct', col_transformer),
                       ('xbg', xgb)])

params = {'xbg__n_estimators': np.linspace(400,800,10,dtype=int),
          'xbg__max_depth': [2,3,4,5,6,7],
          'xbg__eta': np.linspace(0.1,0.2,10)
         }

gs = GridSearchCV(pipe, param_grid= params, scoring=ftwo_scorer, n_jobs=-1, cv=5, verbose= 3)

g_search = gs.fit(X_train, y_train)
gs_model = g_search.best_estimator_
ml, df1 = model_scores('XGBClassifier', gs_model, X_train, y_train, model_list= [], notes = 'Best Estimator from GS')
print(g_search.best_params_)
df1


Fitting 5 folds for each of 600 candidates, totalling 3000 fits
{'xbg__eta': 0.1, 'xbg__max_depth': 2, 'xbg__n_estimators': 400}


,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,XGBClassifier,5.0,50.46,80.44,72.82,32.37,83.65,20.26,Best Estimator from GS


In [ ]:
def model_scores(model_name, model, X, y, model_list= [], cv= True, notes = ''):
    if cv:
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)
        scoring = {'f2': make_scorer(fbeta_score, beta=2),
                   'recall': 'recall',
                   'accuracy':'accuracy',
                   'f1':'f1',
                   'roc_auc':'roc_auc',
                   'precision': 'precision',
                   'log_loss': 'neg_log_loss'}
        scores = cross_validate(model,
                               X,
                               y,
                               scoring = scoring,
                               cv=skf,
                               n_jobs=-1)
        time = round(scores['fit_time'].mean() + scores['score_time'].mean(),0)
        f2 = round(scores['test_f2'].mean(),4)*100
        recall = round(scores['test_recall'].mean(),4)*100
        accuracy = round(scores['test_accuracy'].mean(),4)*100
        f1 = round(scores['test_f1'].mean(),4)*100
        roc_auc = round(scores['test_roc_auc'].mean(),4)*100
        precision = round(scores['test_precision'].mean(),4)*100
        log_loss = round(scores['test_neg_log_loss'].mean(),4)*100
        model_list.append([model_name, time, f2, recall, accuracy, f1, roc_auc, precision, log_loss, notes])
        df = pd.DataFrame(model_list, columns=['model', 'time (in s)', 'f2', 'recall', 'accuracy', 'f1', 'roc_auc', 'precision', 'log_loss', 'notes'])
        return model_list, df
    else:
        y_preds = model.predict(X)
        f2 = fbeta_score(y,y_preds,beta=2)
        recall = recall_score(y,y_preds)
        accuracy = accuracy_score(y,y_preds)
        f1 = f1_score(y,y_preds)
        roc_auc = roc_auc_score(y,y_preds)
        precision = precision_score(y,y_preds)
        log_loss = log_loss(y,y_preds)
        model_list.append([model_name, f2, recall, accuracy, f1, roc_auc, precision, log_loss, notes])
        df = pd.DataFrame(model_list, columns=['model', 'f2', 'recall', 'accuracy', 'f1', 'roc_auc', 'precision', 'log_loss', 'notes'])
        return model_list, df
    

### Final Model


In [22]:
from sklearn.metrics import accuracy_score, precision_score, log_loss
model_list=[]
for model in [dummy_model, logreg_model, rfc_model, xgb_model, xgb_weighted, xgb_smote, xgb_adasyn, rand_model, gs_model]:
    y_preds = model.predict(X_test)
    f2 = fbeta_score(y_test,y_preds,beta=2)
    recall = recall_score(y_test,y_preds)
    accuracy = accuracy_score(y_test,y_preds)
    f1 = f1_score(y_test,y_preds)
    roc_auc = roc_auc_score(y_test,y_preds)
    precision = precision_score(y_test,y_preds)
    logloss = log_loss(y_test,model.predict_proba(X_test))
    model_list.append([f2, recall, accuracy, f1, roc_auc, precision, logloss])
df = pd.DataFrame(model_list, columns=['f2', 'recall', 'accuracy', 'f1', 'roc_auc', 'precision', 'log_loss'])

C:\Users\ronlo\anaconda3\envs\ac-env\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TypeError: 'numpy.float64' object is not callable

In [23]:
recall_score(y_test, gs_model.predict(X_test))

0.8087457952907257

In [25]:
log_loss(y_test,gs_model.predict_proba(X_test))

TypeError: 'numpy.float64' object is not callable